## _*Using Qiskit Aqua algorithms, a how to guide*_

This notebook demonstrates how to use the `Qiskit Aqua` library to invoke a specific algorithm and process the result.

Further information is available for the algorithms in the github repo aqua/readme.md

In [1]:
from qiskit_aqua import Operator, run_algorithm
from qiskit_aqua.input import get_input_instance

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from a previously computed Hamiltonian for simplicity

In [2]:
pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.052373245772859}, "label": "II"},
              {"coeff": {"imag": 0.0, "real": 0.39793742484318045}, "label": "ZI"},
              {"coeff": {"imag": 0.0, "real": -0.39793742484318045}, "label": "IZ"},
              {"coeff": {"imag": 0.0, "real": -0.01128010425623538}, "label": "ZZ"},
              {"coeff": {"imag": 0.0, "real": 0.18093119978423156}, "label": "XX"}
              ]
}

qubitOp = Operator.load_from_dict(pauli_dict)

We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. Now the complete params can be passed to the algorithm and run. The result is a dictionary.

In [3]:
algorithm_cfg = {
    'name': 'ExactEigensolver',
}

params = {
    'algorithm': algorithm_cfg
}
algo_input = get_input_instance('EnergyInput')
algo_input.qubit_op = qubitOp
result = run_algorithm(params,algo_input)
print(result)

{'eigvals': array([-1.85727503-7.00389617e-17j]), 'eigvecs': array([[-1.38777878e-16+2.45029691e-17j,  7.22856695e-01+6.81936898e-01j,
        -8.11307233e-02-7.65380388e-02j, -2.22044605e-16+5.55111512e-17j]]), 'energy': -1.857275030202382, 'wavefunction': array([[-1.38777878e-16+2.45029691e-17j,  7.22856695e-01+6.81936898e-01j,
        -8.11307233e-02-7.65380388e-02j, -2.22044605e-16+5.55111512e-17j]]), 'energies': array([-1.85727503])}


Now we want VQE and so change it and add its other configuration parameters. VQE also needs and optimizer and variational form. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

In [4]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'L_BFGS_B',
    'maxfun': 1000
}

var_form_cfg = {
    'name': 'RYRZ',
    'depth': 3,
    'entanglement': 'linear'
}

params = {
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg,
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(params,algo_input)
print(result)

{'eigvals': array([-1.85727503]), 'opt_params': array([ 3.14152283, -1.86138772,  2.14320588,  2.93799407, -0.28074829,
       -2.77067919, -1.20632283,  0.59925957,  3.14159265,  0.82165996,
        1.00013921, -1.75350817,  2.9702609 , -1.17088041,  0.32988214,
       -2.99356035]), 'eigvecs': array([[-1.98163691e-06-7.88682378e-06j, -8.93429639e-01-4.35136014e-01j,
         1.00272726e-01+4.88416830e-02j,  2.96950529e-06-1.76810183e-06j]]), 'energy': -1.8572750301062404, 'eval_count': 374, 'eval_time': 18.833309173583984}
